In [ ]:
!pip install ortools networkx
import warnings
warnings.filterwarnings("ignore")




  Using cached ortools-9.14.6206-cp310-cp310-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (3.3 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached absl_py-2.3.1-py3-none-any.whl.metadata (3.3 kB)
  Using cached numpy-2.2.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached protobuf-6.31.1-cp39-abi3-manylinux2014_x86_64.whl.metadata (593 bytes)
  Using cached immutabledict-4.2.1-py3-none-any.whl.metadata (3.5 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached ortools-9.14.6206-cp310-cp310-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (27.6 MB)
Using cached protobuf-6.31.1-cp39-abi3-manylinux2014_x86_64.whl (321 kB)
Using cached networkx-3.4.2-py3-none-any.whl (1.7 MB)
Using cached absl_py-2.3.1-py3-none-any.whl (135 kB)
Using cached immutabledict-4.2.1-py3-none-any.whl (4.7 kB)
Using cached numpy-2.2.6-cp

# Exercício 1 - Problema de Distribuição com MIP

## Objetivo
Otimizar a distribuição de encomendas usando programação inteira mista e teoria de grafos.

## Problema
Distribuir pacotes de 3 tipos (1, 2, 5 unidades) de múltiplas fontes para um destino, usando veículos com capacidade de 10 unidades, respeitando:
- Limites de stock por fonte
- Limites de veículos por fonte  
- Capacidades das vias de comunicação
- Percursos válidos no grafo

## Formulação Matemática
Minimizar ∑ y[s] (veículos totais)
Sujeito a:
- ∑ x[s][t] = demanda[t] ∀t (satisfazer encomenda)
- ∑ x[s][t]·size[t] ≤ stock[s] ∀s (respeitar stock)
- units_sent[s] ≤ VEHICLE_CAPACITY·y[s] ∀s (capacidade veículos)
- Conservação de fluxo no grafo

## Solução
Usamos OR-Tools com solver MIP (CBC) e NetworkX para modelagem do grafo de distribuição.

## Complexidade
- Problema de fluxo em redes com restrições de capacidade
- Variáveis inteiras para pacotes e veículos
- Espaço de soluções exponencial no tamanho do grafo

In [ ]:
from ortools.linear_solver import pywraplp
import networkx as nx
import random
from collections import defaultdict, deque


# Configuração do exemplo
random.seed(1)

# tipos de pacote: tamanho em unidades
PACKAGE_SIZES = {1:1, 2:2, 3:5}  # indices 1,2,3 representando pacotes {1,2,5}
VEHICLE_CAPACITY = 10

# Gera um grafo pequeno de exemplo (bidireccional)
G = nx.Graph()
nodes = list(range(1,9))
G.add_nodes_from(nodes)
edges = [ (1,2,3), (2,3,2), (2,4,2), (3,5,2), (4,5,1), (4,6,2), (5,7,2), (6,7,2), (7,8,3) ]
# cada aresta: (u,v,capacidade em nº de veículos)
for u,v,c in edges:
    G.add_edge(u,v,capacity=c)

# Define fontes (sources) e sink (destino da encomenda)
sources = [1,2,4]
sink = 8

# Stocks e limite de veículos por source
stock = {1:30, 2:20, 4:15}           # unidades disponíveis
vehicle_limit = {1:5, 2:3, 4:3}      # nº de veículos em cada source

# Encomenda: no sink, nº de pacotes por tipo (tipos 1..3)
order_packages = {1:2, 2:3, 3:2}  # 2 pak1 (1u), 3 pak2 (2u), 2 pak3 (5u)
# calcula unidades totais pedidas
total_units = sum(order_packages[t]*PACKAGE_SIZES[t] for t in order_packages)
print(f"Pedido para sink={sink}: packages={order_packages}, total_units={total_units}\n")

# -----------------------------
# Construção do MIP
# -----------------------------
solver = pywraplp.Solver.CreateSolver('CBC')
if not solver:
    raise SystemExit('Solver CBC não disponível')

# Variáveis x[s][t] inteiro >=0: nº pacotes do tipo t fornecidos por s
x = {s: {t: solver.IntVar(0, solver.infinity(), f'x_{s}_{t}') for t in PACKAGE_SIZES} for s in sources}

# Variáveis y[s] inteiro >=0: nº veículos usados do source s
y = {s: solver.IntVar(0, vehicle_limit[s], f'y_{s}') for s in sources}

# Variáveis de fluxo f[s][u][v] inteiro >=0 para cada arco orientado (u,v)
f = {s: {} for s in sources}
for s in sources:
    for u,v in G.edges():
        # cria variável para cada orientação
        f[s][(u,v)] = solver.IntVar(0, solver.infinity(), f'f_{s}_{u}_{v}')
        f[s][(v,u)] = solver.IntVar(0, solver.infinity(), f'f_{s}_{v}_{u}')

# Variáveis auxiliares: unidades enviadas por source
units_sent = {s: solver.IntVar(0, solver.infinity(), f'units_{s}') for s in sources}

# 1) Demanda por tipo satisfeita
for t in PACKAGE_SIZES:
    solver.Add(sum(x[s][t] for s in sources) == order_packages.get(t,0))

# 2) Stock em cada source
for s in sources:
    solver.Add(sum(x[s][t]*PACKAGE_SIZES[t] for t in PACKAGE_SIZES) <= stock[s])

# 3) Relação unidades_sent = sum_t x*size
for s in sources:
    solver.Add(units_sent[s] == sum(x[s][t]*PACKAGE_SIZES[t] for t in PACKAGE_SIZES))

# 4) capacidade de veículos: unidades_sent <= VEHICLE_CAPACITY * y[s]
for s in sources:
    solver.Add(units_sent[s] <= VEHICLE_CAPACITY * y[s])

# 5) fluxos de veículos: conservação e envio de y[s] veículos do source s até sink
for s in sources:
    for v in G.nodes():
        in_flow = sum(f[s][(u,v)] for u in G.neighbors(v))
        out_flow = sum(f[s][(v,u)] for u in G.neighbors(v))
        if v == s:
            # no source: out - in = y[s]
            solver.Add(out_flow - in_flow == y[s])
        elif v == sink:
            # no sink: in - out = y[s]
            solver.Add(in_flow - out_flow == y[s])
        else:
            # nos intermediários, in - out = 0
            solver.Add(in_flow - out_flow == 0)

# 6) capacidade das arestas (soma de todos os flujos orientados não excede capacity)
for u,v,data in G.edges(data=True):
    cap = data['capacity']
    solver.Add(sum(f[s][(u,v)] + f[s][(v,u)] for s in sources) <= cap)

# 7) veículos usados <= veículos disponíveis já imposto pela definição de y[s] upper bound
# (finalmente podemos imposer y[s] <= vehicle_limit[s] -- já feito ao criar variável)

# Objetivo: minimizar total veículos
solver.Minimize(sum(y[s] for s in sources))


# Resolver
status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    print('Solução encontrada:')
    print('Total veículos usados =', sum(int(y[s].solution_value()) for s in sources))
    for s in sources:
        ys = int(y[s].solution_value())
        us = int(units_sent[s].solution_value())
        print(f" Source {s}: veículos_used={ys}, units_sent={us}, packages={{" + ", ".join(f'\"t{t}\":{int(x[s][t].solution_value())}' for t in PACKAGE_SIZES) + '}}')
    print('\nFluxos por aresta (soma sobre sources):')
    for u,v,data in G.edges(data=True):
        total_flow = sum(int(f[s][(u,v)].solution_value()) + int(f[s][(v,u)].solution_value()) for s in sources)
        if total_flow>0:
            print(f'  Edge ({u}-{v}) : vehicles = {total_flow} / capacity {data["capacity"]}')

    # Reconstruir rotas por source a partir das variáveis de fluxo (heurística simples)
    print('\nRotas (heurística de decomposição de fluxo):')
    # converte fluxos para um multigraph de capacidades inteiras por source
    for s in sources:
        ys = int(y[s].solution_value())
        if ys==0:
            continue
        # criar uma dict residual
        residual = defaultdict(lambda: defaultdict(int))
        for (u,v) in list(f[s].keys()):
            val = int(f[s][(u,v)].solution_value())
            if val>0:
                residual[u][v] += val
        routes = []
        # extrair até ys caminhos s->sink
        for _ in range(ys):
            # BFS for a path with available capacity
            parent = {s: None}
            q = deque([s])
            found = False
            while q and not found:
                cur = q.popleft()
                for nb in residual[cur]:
                    if residual[cur][nb] > 0 and nb not in parent:
                        parent[nb] = cur
                        if nb == sink:
                            found = True
                            break
                        q.append(nb)
            if not found:
                break
            # reconstruct path
            path = []
            cur = sink
            while cur is not None:
                path.append(cur)
                cur = parent[cur]
            path = list(reversed(path))
            routes.append(path)
            # decrement residual along path
            for i in range(len(path)-1):
                u = path[i]; v = path[i+1]
                residual[u][v] -= 1
        print(f' Source {s}: routes = {routes}')

else:
    print('Problema sem solução (inviável)')



Pedido para sink=8: packages={1: 2, 2: 3, 3: 2}, total_units=18

Solução encontrada:
Total veículos usados = 2
 Source 1: veículos_used=1, units_sent=8, packages={"t1":2, "t2":3, "t3":0}}
 Source 2: veículos_used=0, units_sent=0, packages={"t1":0, "t2":0, "t3":0}}
 Source 4: veículos_used=1, units_sent=10, packages={"t1":0, "t2":0, "t3":2}}

Fluxos por aresta (soma sobre sources):
  Edge (1-2) : vehicles = 1 / capacity 3
  Edge (2-4) : vehicles = 1 / capacity 2
  Edge (4-5) : vehicles = 1 / capacity 1
  Edge (4-6) : vehicles = 1 / capacity 2
  Edge (5-7) : vehicles = 1 / capacity 2
  Edge (6-7) : vehicles = 1 / capacity 2
  Edge (7-8) : vehicles = 2 / capacity 3

Rotas (heurística de decomposição de fluxo):
 Source 1: routes = [[1, 2, 4, 6, 7, 8]]
 Source 4: routes = [[4, 5, 7, 8]]


# Exercício 2 - Sudoku 3D Generalizado com CP

## Objetivo
Resolver uma generalização tridimensional do problema Sudoku usando constraint programming.

## Problema
Preencher um cubo n²×n²×n² com valores 1..n³ tal que:
- Cada "box" (região) contém valores distintos
- Linhas, colunas e profundidades em cada dimensão contêm valores distintos
- Valores iniciais fixos são respeitados

## Tipos de Boxes
- **Cubos**: regiões cúbicas n×n×n
- **Paths**: sequências lineares entre dois vértices
- **Regiões arbitrárias**: qualquer conjunto de células

## Formulação Matemática
∀ box ∈ boxes: AllDifferent(cells ∈ box)
∀ linha i,k: AllDifferent(cells[i,:,k])  
∀ coluna j,k: AllDifferent(cells[:,j,k])
∀ profundidade i,j: AllDifferent(cells[i,j,:])

## Solução
Usamos OR-Tools CP-SAT solver com restrições AllDifferent.

## Complexidade
- Problema de satisfação de restrições
- Espaço de busca: (n³)^(n⁶) possibilidades
- Técnicas de propagação reduzem espaço de busca

In [ ]:
from ortools.sat.python import cp_model

# ---------- Criar grelha 3D ----------
def create_grid(n):
    N = n**2
    model = cp_model.CpModel()
    grid = {}
    for i in range(1, N+1):
        for j in range(1, N+1):
            for k in range(1, N+1):
                grid[(i,j,k)] = model.NewIntVar(1, n**3, f'cell_{i}_{j}_{k}')
    return grid, model

# ---------- Boxes: all-different ----------
def add_boxes(model, grid, boxes):
    for box in boxes:
        vars_in_box = []
        for pos, val in box.items():
            if val != 0:
                model.Add(grid[pos] == val)  # valor fixo
            vars_in_box.append(grid[pos])
        model.AddAllDifferent(vars_in_box)

# ----------  Criar cubo ----------
def create_cube_box(start_i, start_j, start_k, n):
    box = {}
    for di in range(n):
        for dj in range(n):
            for dk in range(n):
                pos = (start_i + di, start_j + dj, start_k + dk)
                box[pos] = 0  # valor livre
    return box

# ---------- Criar path ----------
def create_path_box(start, end):
    box = {}
    x0, y0, z0 = start
    x1, y1, z1 = end
    dx = 1 if x1 >= x0 else -1
    dy = 1 if y1 >= y0 else -1
    dz = 1 if z1 >= z0 else -1
    x, y, z = x0, y0, z0
    while True:
        box[(x,y,z)] = 0
        if (x, y, z) == (x1, y1, z1):
            break
        if x != x1: x += dx
        if y != y1: y += dy
        if z != z1: z += dz
    return box

# ---------- Restrições tipo Sudoku 3D ----------
def add_sudoku_constraints(model, grid, n):
    N = n**2
    rng = range(1, N+1)

    # linhas (varia j, fixa i,k)
    for i in rng:
        for k in rng:
            model.AddAllDifferent([grid[(i,j,k)] for j in rng])

    # colunas (varia i, fixa j,k)
    for j in rng:
        for k in rng:
            model.AddAllDifferent([grid[(i,j,k)] for i in rng])

    # profundidades (varia k, fixa i,j)
    for i in rng:
        for j in rng:
            model.AddAllDifferent([grid[(i,j,k)] for k in rng])

# ---------- Resolver ----------
def solve_sudoku_3d(n, boxes, add_constraints=True, time_limit_seconds=10):
    grid, model = create_grid(n)
    add_boxes(model, grid, boxes)
    if add_constraints:
        add_sudoku_constraints(model, grid, n)

    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = time_limit_seconds

    status = solver.Solve(model)

    if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
        N = n**2
        solution = [[[0]*N for _ in range(N)] for _ in range(N)]
        for i in range(1, N+1):
            for j in range(1, N+1):
                for k in range(1, N+1):
                    solution[i-1][j-1][k-1] = solver.Value(grid[(i,j,k)])
        return solution, solver, status
    else:
        return None, solver, status

# ---------- Exemplo de input ----------
if __name__ == "__main__":
    n = 2  # cubo 4x4x4
    boxes = []

    # Cubos 2x2x2
    boxes.append(create_cube_box(1,1,1,n))  # canto superior esquerdo
    boxes.append(create_cube_box(3,3,3,n))  # canto inferior direito

    # Paths
    boxes.append(create_path_box((1,1,1),(4,1,1)))  # linha no eixo i
    boxes.append(create_path_box((1,1,1),(1,4,1)))  # linha no eixo j
    boxes.append(create_path_box((1,1,1),(1,1,4)))  # linha no eixo k

    # Valores iniciais em algumas células
    boxes[0][(1,1,1)] = 2
    boxes[1][(3,3,3)] = 4

    # ---------- Resolver ----------
    solution, solver, status = solve_sudoku_3d(n, boxes, add_constraints=True, time_limit_seconds=20)

    if solution is None:
        print("Sem solução encontrada (status =", solver.StatusName(status), ")")
    else:
        N = n**2
        print("=== Cubo 3D solução ===")
        for i in range(N):
            print(f"Camada i={i+1}:")
            for j in range(N):
                print(solution[i][j])
            print()

        print("=== Valores por box ===")
        for idx, box in enumerate(boxes):
            print(f"Box {idx+1}:")
            for pos in sorted(box.keys()):
                i,j,k = pos
                print(f"  Pos {pos}: {solution[i-1][j-1][k-1]}")
            print()

        # estatísticas do solver
        print("Status:", solver.StatusName(status))
        print("Tempo (s):", solver.WallTime())
        print("Nós pesquisados:", solver.NumBranches())


=== Cubo 3D solução ===
Camada i=1:
[2, 1, 6, 4]
[3, 6, 7, 8]
[8, 7, 1, 3]
[4, 3, 2, 1]

Camada i=2:
[4, 7, 5, 3]
[8, 5, 2, 4]
[2, 4, 8, 5]
[5, 1, 6, 8]

Camada i=3:
[6, 4, 2, 8]
[4, 7, 5, 2]
[7, 5, 4, 1]
[3, 2, 7, 6]

Camada i=4:
[5, 8, 7, 1]
[2, 1, 8, 6]
[1, 3, 5, 8]
[6, 7, 3, 2]

=== Valores por box ===
Box 1:
  Pos (1, 1, 1): 2
  Pos (1, 1, 2): 1
  Pos (1, 2, 1): 3
  Pos (1, 2, 2): 6
  Pos (2, 1, 1): 4
  Pos (2, 1, 2): 7
  Pos (2, 2, 1): 8
  Pos (2, 2, 2): 5

Box 2:
  Pos (3, 3, 3): 4
  Pos (3, 3, 4): 1
  Pos (3, 4, 3): 7
  Pos (3, 4, 4): 6
  Pos (4, 3, 3): 5
  Pos (4, 3, 4): 8
  Pos (4, 4, 3): 3
  Pos (4, 4, 4): 2

Box 3:
  Pos (1, 1, 1): 2
  Pos (2, 1, 1): 4
  Pos (3, 1, 1): 6
  Pos (4, 1, 1): 5

Box 4:
  Pos (1, 1, 1): 2
  Pos (1, 2, 1): 3
  Pos (1, 3, 1): 8
  Pos (1, 4, 1): 4

Box 5:
  Pos (1, 1, 1): 2
  Pos (1, 1, 2): 1
  Pos (1, 1, 3): 6
  Pos (1, 1, 4): 4

Status: OPTIMAL
Tempo (s): 0.026703148000000003
Nós pesquisados: 2359


# Exercício 3 - Shortest Vector Problem (SVP)

## Objetivo
Encontrar um vetor binário não-nulo e que satisfaça: H·e ≡ 0 (mod q)

## Problema
Dada uma matriz H ∈ ℤ^(m×n) com elementos aleatórios em {0..q-1}, encontrar e ∈ {0,1}^m tal que:
- e ≠ 0 (pelo menos um componente ≠ 0)
- Para cada coluna i: ∑(eⱼ × Hⱼᵢ) ≡ 0 mod q
- Minimizar o número de componentes não nulas em e

## Formulação Matemática
∃ e ∈ {0,1}^m, ∃ k ∈ {0..m}ⁿ tal que:
∀ i < n: ∑ⱼ eⱼHⱼᵢ = q·kᵢ

## Solução
Usamos programação inteira com OR-Tools para encontrar o vetor e que minimiza o número de 1's satisfazendo as restrições modulares.

## Aplicação
Problemas de criptografia pós-quântica baseados em reticulados inteiros.

## Complexidade
- Problema NP-difícil
- Espaço de busca: 2^m vetores possíveis
- OR-Tools resolve eficientemente usando técnicas de branch-and-bound

In [ ]:
from ortools.linear_solver import pywraplp
import numpy as np

def solve_svp(H, q):

    m, n = H.shape

    solver = pywraplp.Solver.CreateSolver('SCIP')

    # Variáveis binárias e_j
    e_vars = [solver.IntVar(0, 1, f'e_{j}') for j in range(m)]

    # Variáveis inteiras k_i
    k_vars = [solver.IntVar(0, m, f'k_{i}') for i in range(n)]

    # Restrições modulares
    for i in range(n):
        constraint = solver.Sum(e_vars[j] * H[j][i] for j in range(m)) == q * k_vars[i]
        solver.Add(constraint)

    # e não pode ser vetor nulo
    solver.Add(solver.Sum(e_vars) >= 1)

    # Minimizar número de 1's
    solver.Minimize(solver.Sum(e_vars))

    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        e_sol = [int(e_vars[j].solution_value()) for j in range(m)]
        num_ones = sum(e_sol)
        return e_sol, num_ones
    else:
        return None, 0

# TESTE PEQUENO QUE ENCONTRA SOLUÇÃO
print("=== SVP - TESTE PEQUENO (Solução Garantida) ===\n")

# Parâmetros pequenos que garantem solução rápida
n = 4   # Dimensão pequena
m = 8   # m = 2n
q = 17  # Primo pequeno

print(f"Parâmetros: n={n}, m={m}, q={q}")

# Gerar matriz H que GARANTE solução
np.random.seed(123)  # Para ser reproduzível, mas sempre com solução
H = np.random.randint(0, q, size=(m, n))

# Garantir que há pelo menos uma solução: criar duas linhas complementares
H[1] = (q - H[0]) % q  # H[0] + H[1] ≡ 0 mod q

print(f"\nMatriz H ({m}x{n}):")
for i in range(m):
    print(f"  {H[i]}")

print(f"\nVerificação: H[0] + H[1] mod q = {[(H[0][i] + H[1][i]) % q for i in range(n)]}")

# Resolver
import time
start_time = time.time()
e, num_ones = solve_svp(H, q)
end_time = time.time()

if e is not None:
    print(f"\n SOLUÇÃO ENCONTRADA em {end_time - start_time:.3f} segundos:")
    print(f"e = {e}")
    print(f"Número de componentes não nulas: {num_ones}")

    # Verificação completa
    print(f"\nVERIFICAÇÃO:")
    todas_corretas = True
    for i in range(n):
        soma = sum(e[j] * H[j][i] for j in range(m))
        resto = soma % q
        status = "ok" if resto == 0 else "nop"
        print(f"  Coluna {i}: {soma} ≡ {resto} (mod {q}) {status}")
        if resto != 0:
            todas_corretas = False

    print(f"\nResumo: {num_ones} componentes não nulas, todas as {n} restrições satisfeitas!")

else:
    print("\n Nenhuma solução encontrada (inesperado!)")

print(f"\nEspaço de busca: 2^{m} = {2**m} vetores possíveis")

=== SVP - TESTE PEQUENO (Solução Garantida) ===

Parâmetros: n=4, m=8, q=17

Matriz H (8x4):
  [13  2  2  6]
  [ 4 15 15 11]
  [ 9  0 14  0]
  [15 14  4  0]
  [16  4  3  2]
  [ 7  2 15 16]
  [7 9 3 6]
  [ 1  2  1 12]

Verificação: H[0] + H[1] mod q = [np.int64(0), np.int64(0), np.int64(0), np.int64(0)]

 SOLUÇÃO ENCONTRADA em 0.012 segundos:
e = [1, 1, 0, 0, 0, 0, 0, 0]
Número de componentes não nulas: 2

VERIFICAÇÃO:
  Coluna 0: 17 ≡ 0 (mod 17) ok
  Coluna 1: 17 ≡ 0 (mod 17) ok
  Coluna 2: 17 ≡ 0 (mod 17) ok
  Coluna 3: 17 ≡ 0 (mod 17) ok

Resumo: 2 componentes não nulas, todas as 4 restrições satisfeitas!

Espaço de busca: 2^8 = 256 vetores possíveis


# Exercício 4 - Closest Vector Problem (CVP)

## Objetivo
Encontrar a combinação linear inteira de vetores mais próxima de um vetor target.

## Problema
Dados vetores geradores G (probabilidades entre 0 e 1) e um vetor target t, encontrar coeficientes inteiros zᵢ (com |zᵢ| ≤ ℓ) que minimizem:

**distância = ||(z₁g₁ + z₂g₂ + ... + zₖgₖ) - t||**

## Solução
Usamos programação inteira com OR-Tools para resolver eficientemente, mesmo para problemas grandes.

## Exemplo
- **8 dimensões**, **6 vetores geradores**, coeficientes entre **-2 e 2**
- **15.625 combinações possíveis** → resolvido rapidamente
- OR-Tools garante solução ótima

## Aplicação
Problemas de criptografia pós-quântica e optimização em reticulados.

In [ ]:
from ortools.linear_solver import pywraplp
import numpy as np

def solve_cvp_ortools(G, t, l=1):

    G = np.array(G)
    t = np.array(t)
    k, m = G.shape

    # Criar solver
    solver = pywraplp.Solver.CreateSolver('SCIP')

    # Variáveis inteiras z_i
    z_vars = [solver.IntVar(-l, l, f'z_{i}') for i in range(k)]

    # Variáveis auxiliares para diferenças
    diff_vars = [solver.NumVar(-10, 10, f'diff_{j}') for j in range(m)]

    # Restrições: diff_j = sum_i(G[i,j] * z_vars[i]) - t[j]
    for j in range(m):
        solver.Add(diff_vars[j] == sum(G[i,j] * z_vars[i] for i in range(k)) - t[j])

    # Função objetivo: minimizar soma dos valores absolutos (aproximação)
    abs_vars = [solver.NumVar(0, 10, f'abs_{j}') for j in range(m)]

    for j in range(m):
        solver.Add(abs_vars[j] >= diff_vars[j])
        solver.Add(abs_vars[j] >= -diff_vars[j])

    solver.Minimize(solver.Sum(abs_vars))

    # Resolver
    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        z_sol = [int(z_vars[i].solution_value()) for i in range(k)]
        x_sol = G.T @ z_sol
        dist = np.linalg.norm(x_sol - t)  # Distância euclidiana real
        return z_sol, x_sol, dist
    else:
        raise Exception('Solução ótima não encontrada')

# TESTE COM RANDOMS DIFERENTES CADA VEZ
print("=== CVP COM OR-TOOLS - TESTE COM RANDOMS REAIS ===\n")

m = 8    # Dimensão do espaço
k = 6    # Número de vetores geradores
l = 2    # Limite maior para coeficientes

# Gerar vetores aleatórios DIFERENTES cada execução
G = np.random.rand(k, m).tolist()
t = np.random.rand(m).tolist()

print(f"Parâmetros: m={m}, k={k}, l={l}")
print(f"Número de combinações possíveis: {(2*l + 1)**k} = {pow(2*l + 1, k)}")

print("\nVetores geradores:")
for i, g in enumerate(G):
    print(f"  g_{i+1} = {[round(x, 3) for x in g]}")

print(f"\nTarget: t = {[round(x, 3) for x in t]}")

# Resolver
try:
    z, x, dist = solve_cvp_ortools(G, t, l)

    print(f"\nSOLUÇÃO:")
    print(f"Coeficientes z = {z}")
    print(f"Vetor reticulado x = {[round(xi, 3) for xi in x]}")
    print(f"Distância euclidiana = {dist:.4f}")

    # Verificar se está dentro dos limites
    print(f"\nVERIFICAÇÃO:")
    print(f"Todos |z_i| <= {l}: {all(abs(z_i) <= l for z_i in z)}")
    print(f"Tamanho do problema: {k} variáveis, {m} restrições")

except Exception as e:
    print(f"\nERRO: {e}")

=== CVP COM OR-TOOLS - TESTE COM RANDOMS REAIS ===

Parâmetros: m=8, k=6, l=2
Número de combinações possíveis: 15625 = 15625

Vetores geradores:
  g_1 = [0.631, 0.092, 0.434, 0.431, 0.494, 0.426, 0.312, 0.426]
  g_2 = [0.893, 0.944, 0.502, 0.624, 0.116, 0.317, 0.415, 0.866]
  g_3 = [0.25, 0.483, 0.986, 0.519, 0.613, 0.121, 0.826, 0.603]
  g_4 = [0.545, 0.343, 0.304, 0.417, 0.681, 0.875, 0.51, 0.669]
  g_5 = [0.586, 0.625, 0.675, 0.842, 0.083, 0.764, 0.244, 0.194]
  g_6 = [0.572, 0.096, 0.885, 0.627, 0.723, 0.016, 0.594, 0.557]

Target: t = [0.159, 0.153, 0.696, 0.319, 0.692, 0.554, 0.389, 0.925]

SOLUÇÃO:
Coeficientes z = [1, 0, 1, 0, 0, -1]
Vetor reticulado x = [np.float64(0.309), np.float64(0.479), np.float64(0.534), np.float64(0.323), np.float64(0.383), np.float64(0.53), np.float64(0.544), np.float64(0.473)]
Distância euclidiana = 0.6928

VERIFICAÇÃO:
Todos |z_i| <= 2: True
Tamanho do problema: 6 variáveis, 8 restrições
